In [ ]:
def parse_input(input_text):
    sections = input_text.strip().split("\n\n")
    map_data = sections[0].split("\n")
    moves = sections[1].replace("\n", "")

    warehouse = []
    robot_position = None
    boxes = set()
    walls = set()

    for r, row in enumerate(map_data):
        warehouse.append(row)
        for c, char in enumerate(row):
            if char == "@":
                robot_position = (r, c)
            elif char == "O":
                boxes.add((r, c))
            elif char == "#":
                walls.add((r, c))

    return warehouse, robot_position, boxes, walls, moves

def move_robot(robot, direction):
    dr, dc = 0, 0
    if direction == "^":
        dr = -1
    elif direction == "v":
        dr = 1
    elif direction == "<":
        dc = -1
    elif direction == ">":
        dc = 1
    return robot[0] + dr, robot[1] + dc

def simulate(warehouse, robot, boxes, walls, moves):
    for move in moves:
        next_pos = move_robot(robot, move)
        if next_pos in walls:
            continue  # Robot cannot move into a wall

        if next_pos in boxes:
            # Calculate box's next position
            box_next_pos = move_robot(next_pos, move)
            if box_next_pos in walls or box_next_pos in boxes:
                continue  # Box cannot be pushed

            # Move the box
            boxes.remove(next_pos)
            boxes.add(box_next_pos)

        # Move the robot
        robot = next_pos

    return robot, boxes

def calculate_gps_coordinates(boxes):
    return sum(100 * r + c for r, c in boxes)

# Read input from file
with open("/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_15/input_15_12_2024.txt", "r") as file:
    input_text = file.read()

# Parse the input
warehouse, robot, boxes, walls, moves = parse_input(input_text)

# Simulate the robot's movements
robot, boxes = simulate(warehouse, robot, boxes, walls, moves)

# Calculate the sum of GPS coordinates
result = calculate_gps_coordinates(boxes)
print("Sum of GPS coordinates:", result)


Sum of GPS coordinates: 1430987


In [ ]:
import numpy as np


class Grid:
    def __init__(self, map):
        self.grid = np.array([list(row) for row in map])
        self.height, self.width = self.grid.shape

        # get info from the grid
        self.robot = tuple(np.argwhere(self.grid == '@')[0])
        self.walls = {tuple(pos) for pos in np.argwhere(self.grid == '#')}
        self.boxes = {tuple(pos) for pos in np.argwhere(self.grid == 'O')}
        self.boxesLeft, self.boxesRight = set(), set()  # needed for part 2

        self.moves = {'^': (-1, 0), 'v': (1, 0), '<': (0, -1), '>': (0, 1)}

    def step(self, direction):
        dx, dy = self.moves[direction]
        next_pos = (self.robot[0] + dx, self.robot[1] + dy)
        if next_pos in self.boxes:
            new_boxes = set()
            moving = True
            current_pos = next_pos

            while moving:
                next_box_pos = (current_pos[0] + dx, current_pos[1] + dy)
                if next_box_pos in self.walls:
                    moving = False
                elif next_box_pos not in self.boxes:
                    new_boxes.add(next_box_pos)
                    break
                else:
                    new_boxes.add(next_box_pos)
                    current_pos = next_box_pos

            if moving:
                for new_box in new_boxes:
                    self.boxes.remove((new_box[0] - dx, new_box[1] - dy))
                self.boxes.update(new_boxes)
                self.robot = next_pos

        elif next_pos not in self.walls:
            self.robot = next_pos

    def double_box_step(self, direction):
        dx, dy = self.moves[direction]
        next_pos = (self.robot[0] + dx, self.robot[1] + dy)

        if next_pos in self.boxesLeft or next_pos in self.boxesRight:
            new_boxes_left = set()
            new_boxes_right = set()

            current_check = set()
            if next_pos in self.boxesLeft:
                current_check.add(("L", next_pos))
                current_check.add(("R", (next_pos[0], next_pos[1] + 1)))
            if next_pos in self.boxesRight:
                current_check.add(("R", next_pos))
                current_check.add(("L", (next_pos[0], next_pos[1] - 1)))

            moving = True
            checked = set()

            while moving:
                free = True
                new_check = set()
                for side, (x, y) in current_check:
                    if (x, y) in checked:
                        continue
                    checked.add((x, y))
                    next_box_pos = (x + dx, y + dy)
                    if next_box_pos in self.walls:
                        moving = False
                        break
                    elif next_box_pos in self.boxesLeft:
                        free = False
                        new_check.add(("L", next_box_pos))
                        new_check.add(("R", (next_box_pos[0], next_box_pos[1] + 1)))
                    elif next_box_pos in self.boxesRight:
                        free = False
                        new_check.add(("R", next_box_pos))
                        new_check.add(("L", (next_box_pos[0], next_box_pos[1] - 1)))

                    if side == 'L':
                        new_boxes_left.add(next_box_pos)
                    if side == 'R':
                        new_boxes_right.add(next_box_pos)

                if free:
                    break
                else:
                    current_check = new_check

            if moving:
                for new_box in new_boxes_left:
                    self.boxesLeft.remove((new_box[0] - dx, new_box[1] - dy))
                self.boxesLeft.update(new_boxes_left)

                for new_box in new_boxes_right:
                    self.boxesRight.remove((new_box[0] - dx, new_box[1] - dy))
                self.boxesRight.update(new_boxes_right)

                self.robot = next_pos

        elif next_pos not in self.walls:
            self.robot = next_pos

    def walk(self, directions, part1=True):
        for direction in directions:
            (self.step if part1 else self.double_box_step)(direction)

    def calculate_gps_sum(self, part1=True):
        boxes = self.boxes if part1 else self.boxesLeft
        return sum(100 * x + y for x, y in boxes)

    def print_grid(self, part1=True):
        printgrid = np.full_like(self.grid, ".", dtype=str)

        if part1:
            for pos, char in {**{pos: '#' for pos in self.walls},
                              **{pos: 'O' for pos in self.boxes},
                              self.robot: '@'}.items():
                printgrid[pos] = char
        else:
            for pos, char in {**{pos: '#' for pos in self.walls},
                              **{pos: '[' for pos in self.boxesLeft},
                              **{pos: ']' for pos in self.boxesRight},
                              self.robot: '@'}.items():
                printgrid[pos] = char

        print("\n".join("".join(row) for row in printgrid))

    def scale_up_map(self):
        # Scale up the map width
        scaled_map = []
        scale_dict = {'#': '##', 'O': '[]', '.': '..', '@': '@.'}
        for row in self.grid:
            scaled_row = ''.join(scale_dict[cell] for cell in row)
            scaled_map.append(scaled_row)

        self.grid = np.array([list(row) for row in scaled_map])
        self.height, self.width = self.grid.shape
        self.robot = tuple(np.argwhere(self.grid == '@')[0])
        self.walls = {tuple(pos) for pos in np.argwhere(self.grid == '#')}
        self.boxesLeft = {tuple(pos) for pos in np.argwhere(self.grid == '[')}
        self.boxesRight = {tuple(pos) for pos in np.argwhere(self.grid == ']')}


with open('/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_15/input_15_12_2024.txt') as f:
    map, directions = f.read().split("\n\n")
    Map = Grid(map.split("\n"))
    # Part One
    Map.walk(directions.replace("\n", ""))
    print("Part One GPS Sum:", Map.calculate_gps_sum())
    print("Based on grid:")
    Map.print_grid()

    # Part two
    print()
    Map.scale_up_map()
    Map.walk(directions.replace("\n", ""), False)
    print("Part Two GPS Sum:", Map.calculate_gps_sum(False))
    print("Based on grid:")
    Map.print_grid(False)

Part One GPS Sum: 1406628
Based on grid:
##################################################
##.O.O...#OO...O.#....#....#O#O..O.OO.....O...OOO#
#OO.O.#.O.O.....OO#...O.OO.O....O.#OO.....OO#.OOO#
#...#.#O..O...OO......O....O.O#..OO....#....O.OOO#
#.O...O..OO#.........O..OOOOOOO........O.....OOO.#
#....OO.O...O#.OO..O..#.OOOO.O#O..#........#.#O.##
#O...O.#....OO#O...OO..OOOOO..OO.............O...#
##..OOO.....#..O.........O......O................#
#OOOOOO@.........O...O..........O....OO..........#
#OO##....O.......O.#...O.#...O...O....#......O#O.#
#...O.........#.....OOO..........#OO.............#
#......O....O.....OO......#......#OO..........OO.#
#......O#......#O.OO...OO.#OO....##O#O.O.....##O.#
#.#....#O......OO....O.OO.O....#.O.#O....O...O.O.#
#...O..O#...#.O.....OO..OOOO....OOO.O......OO..O.#
#...O........O...O.#O#O.O.O.##O....#.......OOOO..#
#.#.......OO....OOOOO...O.O.#OOO...O.......OO..#O#
#O.....#.....#...OO..O.#.OOO.O#OO.....O...O.OOOOO#
#O...OO........#.....O.....OOOO..#....#.O